Search database for all CSV files.

In [ ]:
import os
import pandas as pd
from m0_1_system_variables import test_data_path

Create a directory of source files:

In [ ]:
complete_csv_path_list = []
for root, dirs, files in os.walk(test_data_path):
        for filename in files:
                if filename.lower().endswith('csv'):
                    complete_csv_path_list.append(r"{0}\{1}".format(root, filename))

Identify formatting of source files and import the data to buffer:

In [ ]:
import chardet

data_type = []
# Determine the file encoding
for file_path in complete_csv_path_list:
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    file_encoding = result['encoding']
    data_type.append(file_encoding)

In [ ]:
print(data_type)

In [ ]:
import pandas as pd

# Create an empty list to store the dataframes
df_list = []
vin_list = []
# Iterate through each file path in the list
for file_path in complete_csv_path_list:
    # Read the CSV file into a dataframe
    # Difficulty here: some files are ascii and some files are 'Windows-1252' encoding.
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
        detected_encoding = result['encoding']

    df = pd.read_csv(file_path, delimiter = '\r\n', encoding = detected_encoding, engine = 'python', names = ['Complete Record'])
    df_list.append(df)

In [ ]:
df_list[:2]

In [ ]:
master_import_dataframe = pd.DataFrame(columns=['Complete Records'])

for object in df_list:
    master_import_dataframe = pd.concat([master_import_dataframe, pd.DataFrame({'Complete Records': [object]})], ignore_index=True)


In [ ]:
master_import_dataframe.head()

In [ ]:
master_import_dataframe['Complete Records'][14].head()
test_cell = master_import_dataframe['Complete Records'][14].head()

In [ ]:
#The dataframe is complete, but the CSV is truncated. Something is breaking here.
master_import_dataframe.to_csv('master_dataframe.csv', index=False)

In [ ]:
master_import_dataframe["Complete Records"][24].to_csv('test_single_instance.csv', index=False)

In [ ]:
master_import_dataframe_from_file = pd.read_csv('master_dataframe.csv')

In [ ]:
master_import_dataframe_from_file.head()

pd will export single instances to CSV but does not adequately preserve the entire dataframe. Parquet did not function correctly, so pickle was chosen.

In [ ]:
master_import_dataframe.to_pickle("master_import_dataframe.pkl")